In [15]:
import openai, json

client = openai.OpenAI()
messages = []

In [18]:
def get_weather(city):
    return "33 degress celcius."

FUNCTION_MAP = {"get_weather": get_weather}

In [ ]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "A function to get the weather of a city",
            "parameters": {
                "type": "object",
                "properties":{
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the weather of.",   
                    },
                },
                "required": ["city"],
            },
        },
    },
]

In [ ]:
from ast import arguments
from os import name
from openai.types.chat import ChatCompletionMessage

def process_ai_response(message: ChatCompletionMessage):
    if message.tool_calls > 0:
        messages.append(
            {
                "role": "assistant",
                "content": message.content or "",
                "tool_calls": [
                    {
                        "id": tool_call.id,
                        "type": "function",
                        "function": {
                            "name": tool_call.function.name,
                            "arguments": tool_call.function.arguments,
                        },
                    } 
                    for tool_call in message.tool_calls
                ],
            }
        )

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments
            
            print(f"Calling function: {function_name} with : {arguments}")
            
            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}
                
            function_to_run = FUNCTION_MAP.get(function_name)

            result = function_to_run(**arguments)

            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": function_name,
                "content": result,
            })


In [ ]:
def call_ai():
    response = client.chat.completions.create(
        model="gpt-4o-mini", 
        messages=messages,
        tools=TOOLS,
    )
    process_ai_response(response.choices[0].message)
    print(response)
    message = response.choices[0].message.content
    messages.append({"role": "assistant", "content": message})
    print(f"AI: {message}")

In [14]:
while True:
    message = input("Send a message to the LLM...") 
    if message == "quit" or message == "q":
        break
    else:
        messages.append({"role": "user","content": message})
        print(f"User: {message}")
        call_ai()

User: My name is Noori
AI: Hi, Noori! How can I assist you today?
User: What is my name
AI: Your name is Noori. How can I help you today?
User: I'm from Korea
AI: That's great to know, Noori! Korea has a rich culture and history. What would you like to talk about?
User: what was the first question I asked you and what is the closest island from where I'm from? 
AI: The first question you asked was, "What is my name?" 

As for the closest island to Korea, it depends on where you are in Korea. However, Jeju Island is the largest and most famous island in South Korea, located to the south of the mainland. If you meant an island closer to the northern part of the peninsula, there are smaller islands like Ulleungdo and Dokdo in the East Sea (or Sea of Japan). Let me know if you need more specific information!
